In [1]:
import re
from langchain_community.document_loaders import PyPDFLoader

# 1. Wczytanie dokumentu
file_path = "last_unified_labor_code.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()

# 2. Łączenie wszystkich stron w jeden tekst i proste czyszczenie szumu
full_text = ""
for page in pages:
    content = page.page_content
    # Usuwanie stopki (©Kancelaria Sejmu + data + numer strony)
    # Regex dopasuje np. "©Kancelaria Sejmu s. 5/185" oraz datę pod spodem
    content = re.sub(r"©Kancelaria Sejmu.*s\.\s\d+/\d+", "", content)
    content = re.sub(r"2026-02-03", "", content) # usuwa datę generowania
    full_text += content + "\n"

# 3. Funkcja do podziału na artykuły
def split_into_articles(text):
    # Szuka wzorca: "Art. [liczba i ewentualnie litera]."
    # Używa (?=Art\.) aby zachować "Art." na początku każdego chunka
    pattern = r"(?=Art\.\s+\d+[a-z]*\.)"
    chunks = re.split(pattern, text)
    
    # Usuwa ewentualny pusty pierwszy element i białe znaki
    chunks = [c.strip() for c in chunks if c.strip()]
    return chunks

articles = split_into_articles(full_text)

# podgląd
print(f"Znaleziono {len(articles)} jednostek redakcyjnych (artykułów).")
print("\nPrzykład artykułu 1 (Art. 1):")
print(articles[0][:300])
print("\nPrzykład artykułu ze środka:")
print(articles[100][:300])

Znaleziono 477 jednostek redakcyjnych (artykułów).

Przykład artykułu 1 (Art. 1):
U S T AWA 
z dnia 26 czerwca 1974 r. 
Kodeks pracy1) 
Preambuła (uchylona) 
 
1) Niniejsza ustawa dokonuje w  zakresie swojej regulacji wdrożenia następujących dyrektyw 
Wspólnot Europejskich: 
1) dyrektywy 83/477/EWG z dnia 19 września 1983 r. w sprawie ochrony pracowników przed 
ryzykiem związanym

Przykład artykułu ze środka:
Art. 612. § 1. Odszkodowanie, o  którym mowa w  art. 611, przysługuje 
w wysokości wynagrodzenia pracownika za okres wypowiedzenia. W  przypadku 
rozwiązania umowy o  pracę zawartej na czas określony, odszkodowanie 
przysługuje w wysokości wynagrodzenia za czas, do którego umowa miała trwać, 
nie wi


In [2]:
processed_data = []

for i, chunk in enumerate(articles):
    # 1. Wyciąga numer artykułu z początku tekstu (np. "Art. 1.") ## szuka wzorca na samym początku chunka
    match = re.search(r"Art\.\s+(\d+[a-z]*)", chunk)
    
    if match:
        art_number = f"Art. {match.group(1)}"
    else:
        # jeśli to pierwszy chunk (ten z tytułem i preambułą)
        art_number = "Wstęp/Tytuł"

    # 2. Tworzy słownik z metadanymi
    metadata = {
        "art_id": art_number,
        "source": "Kodeks Pracy",
        "status_date": "2026-02-03",
        "chunk_id": i
    }
    
    # 3. Dodajemy do listy gotowy obiekt
    processed_data.append({
        "content": chunk,
        "metadata": metadata
    })

# test
print(f"Przygotowano {len(processed_data)} obiektów z metadanymi.")
print(f"Przykładowy wpis (indeks 5):")
print(f"Metadane: {processed_data[5]['metadata']}")
print(f"Treść (początek): {processed_data[5]['content'][:100]}...")

Przygotowano 477 obiektów z metadanymi.
Przykładowy wpis (indeks 5):
Metadane: {'art_id': 'Art. 4', 'source': 'Kodeks Pracy', 'status_date': '2026-02-03', 'chunk_id': 5}
Treść (początek): Art. 4. (uchylony)...


In [3]:
from sentence_transformers import SentenceTransformer
import torch

# 1. Wybór modelu (multilingual, 384 wymiary)
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

# Sprawdza czy CUDA (GPU) jest dostępne
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Używam urządzenia: {device.upper()}")

# 2. Ładowanie modelu
model = SentenceTransformer(model_name, device=device)

# 3. TEST: Embeddings Sample (na pierwszym artykule)
sample_text = processed_data[1]['content'] ### Art. 1
sample_vector = model.encode(sample_text)

print(f"\nModel załadowany: {model_name}")
print(f"Rozmiar wektora (wymiary): {len(sample_vector)}")
print(f"Przykładowe wartości (pierwsze 5): {sample_vector[:5]}")


Używam urządzenia: CUDA

Model załadowany: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Rozmiar wektora (wymiary): 384
Przykładowe wartości (pierwsze 5): [-0.06652039  0.18819958 -0.33668807  0.27820015  0.07411028]


# Full Embeddings

In [5]:
# 4. Wektoryzacja całego zbioru (full)
print(f"Generowanie wektorów dla {len(processed_data)} artykułów...")

# Wyciąga samą treść do zakodowania
texts_to_encode = [item['content'] for item in processed_data]

# Generuje wektory
all_embeddings = model.encode(texts_to_encode, show_progress_bar=True)

# Dodaje wektory do struktury danych
for i, item in enumerate(processed_data):
    item['embedding'] = all_embeddings[i].tolist() ### zmienia na listę żeby Qdrant to przyjął

print(f"Wszystkie artykuły zostały zamienione na wektory.")

Generowanie wektorów dla 477 artykułów...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Wszystkie artykuły zostały zamienione na wektory.


In [11]:
#### próbka kontrolna
idx = 15
print(f"Sprawdzanie wpisu o indeksie {idx}:")
print(f"Metadane: {processed_data[idx]['metadata']}")
print(f"Treść (fragment): {processed_data[idx]['content'][:150]}...")
print(f"Rozmiar wektora: {len(processed_data[idx]['embedding'])}")
print(f"Wektor (początek): {processed_data[idx]['embedding'][:5]}...")

Sprawdzanie wpisu o indeksie 15:
Metadane: {'art_id': 'Art. 112', 'source': 'Kodeks Pracy', 'status_date': '2026-02-03', 'chunk_id': 15}
Treść (fragment): Art. 112. Pracownicy mają równe prawa z  tytułu jednakowego wypełniania 
takich samych obowiązków; dotyczy to w  szczególności równego traktowania 
mę...
Rozmiar wektora: 384
Wektor (początek): [-0.00595830800011754, 0.29976141452789307, -0.24657131731510162, -0.023757465183734894, 0.1003313809633255]...


In [13]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

# 1. Połączenie z Qdrantem
client = QdrantClient(host="localhost", port=6333)

COLLECTION_NAME = "labor_code_pl"

# 2. Tworzenie kolekcji (jeśli nie istnieje)
if not client.collection_exists(collection_name=COLLECTION_NAME):
    print(f"Tworzenie kolekcji: {COLLECTION_NAME}")
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE),
    )
else:
    print(f"Kolekcja '{COLLECTION_NAME}' już istnieje.")

# 3. Przygotowanie "Punktów" (Points) do wysłania
points = []
for i, item in enumerate(processed_data):
    points.append(
        PointStruct(
            id=i, 
            vector=item['embedding'], 
            payload={
                "content": item['content'],
                **item['metadata'] ## rozpakowuje metadane (art_id, source itp.)
            }
        )
    )

# 4. Wysyłka (Upsert)
print(f"Wysyłanie {len(points)} punktów do Qdranta...")
client.upsert(collection_name=COLLECTION_NAME, points=points)
print("Zakończono.")

C:\Users\Greg Z\AppData\Local\Temp\ipykernel_34512\3236154716.py:5: UserWarning: Qdrant client version 1.13.0 is incompatible with server version 1.16.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  client = QdrantClient(host="localhost", port=6333)


Tworzenie kolekcji: labor_code_pl
Wysyłanie 477 punktów do Qdranta...
Zakończono.
